# Biosemi Active II recorded file streaming

Resample and Save

In [ ]:
# import mne
# import numpy as np
# from mne.channels import make_standard_montage
# import matplotlib.pyplot as plt
# from mne.datasets import eegbci

# raw = mne.io.read_raw_bdf("datasets/Testdata3.bdf")

# eegbci.standardize(raw)  # set channel names
# montage = make_standard_montage("biosemi64")

# raw.set_montage(montage, on_missing='ignore')

# raw_eeg = raw.copy().resample(512)

# raw_eeg.save("datasets/biosemi_SSVEP.fif", overwrite=True)

Load data for stream

In [ ]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci


raw_eeg = mne.io.read_raw_fif("datasets/biosemi_SSVEP.fif")
# select_ch = ['O1','Oz','PO3','PO4','POz','Pz']
# raw_eeg = raw_eeg.pick(select_ch)

Opening raw data file datasets/biosemi_SSVEP.fif...
    Range : 0 ... 100863 =      0.000 ...   196.998 secs
Ready.


C:\Users\pipo_\AppData\Local\Temp\ipykernel_33672\245558524.py:8: RuntimeWarning: This filename (datasets/biosemi_SSVEP.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg = mne.io.read_raw_fif("datasets/biosemi_SSVEP.fif")


In [2]:
from mne_realtime import LSLClient, MockLSLStream
from pylsl import StreamInfo, StreamOutlet
import time
import mne

# Assuming raw_eeg is already created
# Create the LSL stream for the raw EEG data
eeg_info = StreamInfo(name='SomSom', type='EEG', channel_count=raw_eeg.info['nchan'],
                      nominal_srate=raw_eeg.info['sfreq'], channel_format='float32')
eeg_outlet = StreamOutlet(eeg_info)

def send_eeg_samples(raw_eeg):
    """ Send raw EEG data sample-by-sample through LSL. """
    sfreq = raw_eeg.info['sfreq']  # Sampling frequency
    n_samples = len(raw_eeg.times)  # Total number of samples in the raw data
    data, _ = raw_eeg[:, :]  # Get all the EEG data as an array

    for idx in range(n_samples):
        # Extract the single sample data
        sample = data[:, idx]
        # Push the single sample of raw EEG data to the LSL stream
        eeg_outlet.push_chunk(sample.tolist())  # Push the single sample
        # print(f"Sent sample at index {idx}")
        time.sleep(1 / sfreq)  # Maintain the sample rate

if __name__ == '__main__':
    while True:
        # Send raw EEG data sample-by-sample
        send_eeg_samples(raw_eeg)

C:\Users\pipo_\AppData\Local\Temp\ipykernel_33672\3689781721.py:1: FutureWarning: MNE-realtime is discontinued in favor of the Lab Streaming Layer real-time interface availabe in MNE-lsl: https://mne.tools/mne-lsl/
  from mne_realtime import LSLClient, MockLSLStream


KeyboardInterrupt: 